# 📘 **Aplicación de la transformada de Laplace en sistemas de ecuaciones diferenciales**
## 🔍 Introducción
Las ecuaciones diferenciales juegan un papel fundamental en la modelización matemática de fenómenos físicos, biológicos y de ingeniería. En particular, los sistemas de ecuaciones diferenciales lineales aparecen en circuitos eléctricos, dinámica de poblaciones, y control de sistemas mecánicos.

Una herramienta poderosa para resolver estos sistemas es la **Transformada de Laplace**, la cual convierte ecuaciones diferenciales en ecuaciones algebraicas más sencillas de manejar en el dominio de la frecuencia.

En este cuaderno exploraremos cómo aplicar la **Transformada de Laplace** para resolver un sistema de ecuaciones diferenciales de la forma:

$$
\frac{dx}{dt} = 3x + 4y
$$

$$
\frac{dy}{dt} = -4x + 3y
$$

Adicional, compararemos la **solución analítica** obtenida con Laplace con la **solución numérica** usando `solve_ivp` de SciPy. ¡Con elementos interactivos para que experimentes con diferentes valores iniciales! 🚀


### 🎯 **Objetivos de aprendizaje**
🔹 Entender cómo la Transformada de Laplace facilita la resolución de sistemas de ecuaciones diferenciales.  
🔹 Aplicar la Transformada de Laplace para convertir ecuaciones diferenciales en ecuaciones algebraicas.  
🔹 Resolver el sistema de ecuaciones en el dominio de Laplace y luego aplicar la Transformada Inversa.  
🔹 Comparar la solución analítica con una solución numérica usando `solve_ivp`.  
🔹 Permitir la **exploración interactiva** de las soluciones con diferentes condiciones iniciales.  


### ⚡ **Transformada de Laplace**
La **Transformada de Laplace** convierte una ecuación diferencial en una ecuación algebraica en términos de la variable \( s \):

$$
\mathcal{L} \{ f(t) \} = F(s) = \int_0^\infty e^{-st} f(t) dt
$$
Aplicando esta transformación a un sistema de ecuaciones diferenciales, podemos resolverlo más fácilmente en el dominio de \( s \), y luego obtener la solución en el tiempo aplicando la **Transformada Inversa de Laplace**.



In [19]:
# Importar bibliotecas necesarias
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import ipywidgets as widget
import matplotlib.font_manager as fm
from IPython.display import display

In [12]:
!apt-get install fonts-noto

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto is already the newest version (20201225-1build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [23]:
# Soporte emojis
plt.rcParams["font.family"] = "DejaVu Sans"

In [13]:
# Definir las variables simbólicas
t, s = sp.symbols('t s')  # t = tiempo, s = variable de Laplace
X, Y = sp.symbols('X Y')  # Funciones en el dominio de Laplace

In [14]:
# Definir el sistema de ecuaciones diferenciales
x_t, y_t = sp.Function('x')(t), sp.Function('y')(t)
eq1 = sp.Eq(sp.diff(x_t, t), 3*x_t + 4*y_t)
eq2 = sp.Eq(sp.diff(y_t, t), -4*x_t + 3*y_t)

In [15]:
# Aplicar la Transformada de Laplace
X_s, Y_s = sp.symbols('X_s Y_s')
eq1_laplace = sp.laplace_transform(eq1.lhs - eq1.rhs, t, s, noconds=True)
eq2_laplace = sp.laplace_transform(eq2.lhs - eq2.rhs, t, s, noconds=True)

In [26]:
# Función para resolver y graficar interactivamente
def resolver_sistema(x0=1, y0=0):
    # Aplicar condiciones iniciales en el dominio de Laplace
    ICs = {sp.laplace_transform(x_t, t, s, noconds=True): X_s,
           sp.laplace_transform(y_t, t, s, noconds=True): Y_s}

    eq1_s = eq1_laplace.subs(ICs).subs({x_t.subs(t, 0): x0, y_t.subs(t, 0): y0})
    eq2_s = eq2_laplace.subs(ICs).subs({x_t.subs(t, 0): x0, y_t.subs(t, 0): y0})

    # Resolver para X(s) y Y(s)
    sol = sp.solve([eq1_s, eq2_s], (X_s, Y_s))
    X_s_sol, Y_s_sol = sol[X_s], sol[Y_s]

    # Transformada Inversa de Laplace para obtener x(t) y y(t)
    x_t_sol = sp.inverse_laplace_transform(X_s_sol, s, t)
    y_t_sol = sp.inverse_laplace_transform(Y_s_sol, s, t)

    # Definir el sistema para la solución numérica
    def sistema_dif(t, z):
        x, y = z
        return [3*x + 4*y, -4*x + 3*y]

    # Resolver el sistema numéricamente
    t_span = (0, 5)
    t_eval = np.linspace(0, 5, 100)
    sol_numeric = solve_ivp(sistema_dif, t_span, [x0, y0], t_eval=t_eval)

    # Graficar las soluciones
    plt.figure(figsize=(8,5))

    # Solución numérica
    plt.plot(sol_numeric.t, sol_numeric.y[0], label="x(t) numérico", linestyle='dashed', color='blue')
    plt.plot(sol_numeric.t, sol_numeric.y[1], label="y(t) numérico", linestyle='dashed', color='red')

    # Solución analítica
    plt.plot(t_eval, [x_t_sol.subs(t, val).evalf() for val in t_eval], label="x(t) analítico", color='blue')
    plt.plot(t_eval, [y_t_sol.subs(t, val).evalf() for val in t_eval], label="y(t) analítico", color='red')

    # Etiquetas y formato
    plt.xlabel("Tiempo t", fontsize=12, color="white")
    plt.ylabel("Valores de x(t) y y(t)", fontsize=12, color="white")
    plt.title("📈 Solución del sistema de ecuaciones diferenciales", fontsize=14, color="cyan")
    plt.legend()
    plt.grid()
    plt.show()

    # Mostrar soluciones analíticas
    display(sp.Eq(x_t, x_t_sol))
    display(sp.Eq(y_t, y_t_sol))

In [27]:
# Widgets interactivos para modificar condiciones iniciales
x0_slider = widgets.FloatSlider(min=-5, max=5, step=0.1, value=1, description="x(0)")
y0_slider = widgets.FloatSlider(min=-5, max=5, step=0.1, value=0, description="y(0)")

# Interactividad
widgets.interactive(resolver_sistema, x0=x0_slider, y0=y0_slider)

interactive(children=(FloatSlider(value=1.0, description='x(0)', max=5.0, min=-5.0), FloatSlider(value=0.0, de…

### **Resultados y visualización** 📊
Una vez desarrollado y ejecutado el script, obtendremos:

✅ Expresión analítica de la solución obtenida mediante la Transformada de Laplace.  
✅ Solución numérica usando `solve_ivp` de SciPy.  
✅ Gráficos comparativos de la solución analítica y numérica en función del tiempo.  
✅ Interactividad para modificar condiciones iniciales y ver en tiempo real cómo cambia la solución.  

💡 ***¿Qué observas al cambiar los valores iniciales? ¿Cómo afectan a la dinámica del sistema?***  

###  **Conclusiones**
 La Transformada de Laplace es una herramienta poderosa para resolver ecuaciones diferenciales, facilitando su tratamiento en el dominio de la frecuencia.  
 La solución obtenida con Laplace coincide con la solución numérica, validando la técnica utilizada.  
 La interactividad permite explorar la evolución del sistema con diferentes condiciones iniciales, proporcionando una mejor comprensión visual y conceptual.  

Este enfoque es fundamental en áreas como control de sistemas, circuitos eléctricos, y dinámica de poblaciones, donde el análisis de estabilidad y respuesta en el tiempo es clave.  

💡 **¡Experimenta con el código y diviértete aprendiendo! 🚀📈**



### 📚 **Referencias**  
- Boyce, W. E., & DiPrima, R. C. (2017). *Elementary Differential Equations and Boundary Value Problems* (11th ed.). Wiley.  
- Kreyszig, E. (2011). *Advanced Engineering Mathematics* (10th ed.). Wiley.  
- Ogata, K. (2020). *Ingeniería de Control Moderna* (6ta ed.). Pearson.  
- Bronson, R. (2014). *Schaum's Outline of Differential Equations*. McGraw-Hill.  
- Documentation SciPy: https://docs.scipy.org/doc/scipy/reference/integrate.html  

  